In [ ]:
import tables_io
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# tract = 10705
tract = 10463
shear = "0p02"
t = tables_io.read(f"shear_wmom_{shear}_match_{tract}_object_shear.pq")     
t2 = tables_io.read(f"shear_wmom_{shear}_match_{tract}_object_stats.pq")
t['idx'] = np.arange(len(t))
t2['idx'] = np.arange(len(t2))

In [ ]:
merged = t.merge(t2, on='idx')

In [ ]:
in_cell_mask = np.bitwise_and( 
    np.fabs(merged.xCents-100) < 75,
    np.fabs(merged.yCents-100) < 75,
)
in_cell = merged[in_cell_mask]
bright_mask = in_cell.SNRs > 7.5

used = in_cell[bright_mask]

good_mask = used.good
good = used[good_mask]
bad = used[~good_mask]

In [ ]:
print("All Objects:                                   ", len(merged))
print("Usable                                         ", len(in_cell))
print("Used                                           ", len(used))
print("Good                                           ", len(good))
print("Bad                                            ", len(bad))

In [ ]:
def stats(weights, bin_centers):
    w = np.sum(weights)
    mean = np.sum(weights*bin_centers)/w
    deltas = bin_centers - mean
    var = np.sum(weights*deltas*deltas)/w
    std = np.sqrt(var)
    error = std/np.sqrt(w)
    inv_var = 1./(error*error)
    return mean, std, error, inv_var

In [ ]:
bin_edges = np.linspace(-1, 1, 2001)
bin_centers = (bins_edges[1:] + bins_edges[0:-1])/2.

good_delta_g1 = np.histogram(good.delta_g_1, bins=bin_edges)[0]
good_delta_g2 = np.histogram(good.delta_g_2, bins=bin_edges)[0]

good_g2_2p = np.histogram(good.g2_2p, weights=good.n_2p, bins=bin_edges)[0]
good_g2_2m = np.histogram(-1*good.g2_2m, weights=good.n_2m, bins=bin_edges)[0]
good_g1_1p = np.histogram(good.g1_1p, weights=good.n_1p, bins=bin_edges)[0]
good_g1_1m = np.histogram(-1*good.g1_1m, weights=good.n_1m, bins=bin_edges)[0]

bad_g2_2p = np.histogram(bad.g2_2p, weights=bad.n_2p, bins=bin_edges)[0]
bad_g2_2m = np.histogram(-1*bad.g2_2m, weights=bad.n_2m, bins=bin_edges)[0]
bad_g1_1p = np.histogram(bad.g1_1p, weights=bad.n_1p, bins=bin_edges)[0]
bad_g1_1m = np.histogram(-1*bad.g1_1m, weights=bad.n_1m, bins=bin_edges)[0]

stats_delta_g1 = stats(good_delta_g1, bin_centers)
stats_delta_g2 = stats(good_delta_g2, bin_centers)

stats_good_g1 = stats(good_g1_1p + good_g1_1m, bin_centers)
stats_good_g2 = stats(good_g2_2p + good_g2_2m, bin_centers)

stats_bad_g1 = stats(bad_g1_1p + bad_g1_1m, bin_centers)
stats_bad_g2 = stats(bad_g2_2p + bad_g2_2m, bin_centers)


In [ ]:
_ = plt.stairs(good_delta_g1[800:1200], bin_edges[800:1201], label=f"g1: {100*stats_delta_g1[0]:.4f} +- {100*stats_delta_g1[2]:.4f}")
_ = plt.stairs(good_delta_g2[800:1200], bin_edges[800:1201], label=f"g2: {100*stats_delta_g2[0]:.4f} +- {100*stats_delta_g2[2]:.4f}")
_ = plt.axvline(x=0, color='red', linestyle='--', linewidth=2)
_ = plt.legend()

In [ ]:
_ = plt.stairs(
    (good_g1_1p + good_g1_1m)[800:1200], bin_edges[800:1201],
    label=f"g1: {200*stats_good_g1[0]:.4f} +- {200*stats_good_g1[2]:.4f}",
)
_ = plt.stairs(
    (good_g2_2p + good_g2_2m)[800:1200], bin_edges[800:1201],
    label=f"g2: {200*stats_good_g2[0]:.4f} +- {200*stats_good_g2[2]:.4f}",
)
_ = plt.legend()

In [ ]:
_ = plt.stairs(
    (bad_g1_1p + bad_g1_1m)[800:1200], bin_edges[800:1201],
    label=f"g1: {200*stats_bad_g1[0]:.4f} +- {200*stats_bad_g1[2]:.4f}",
)
_ = plt.stairs(
    (bad_g2_2p + bad_g2_2m)[800:1200], bin_edges[800:1201],
    label=f"g2: {200*stats_bad_g2[0]:.4f} +- {200*stats_bad_g2[2]:.4f}",
)
_ = plt.legend()

In [ ]:
stats_bad_g1[3]

In [ ]:
stats_good_g1[3] / stats_bad_g1[3]